# SLP

## 선택 분류 문제의 신경망 처리

선택 분류 문제는 몇 가지 정해진 후보 가운데 하나를 골라 답하는 문제이다. 선택 분류는 후보 항목이 여럿인 탓에 처리 방법이 달라지기는 하지만 기본적으로 Binary Classification과 유사하다.

Binary Classification처럼 신경망이 직접 후보 항목의 번호를 지정한다거나 각 후보 항목의 확률값을 계산하기는 어렵지만, 선택 분류 신경망은 Binary Classification처럼 일단 각 후보 항목에 대한 로그 척도의 상대적 추천 강도, 즉 로짓값을 추정하도록 구성된다.

로짓값은 상대적인 가능성을 로그를 이용해 나타낸 값이다. 선택 분류에서 로짓값의 표현 대상은 각 후보 항목을 답으로 추정할 확률이다.
    
    ex1) A 항목의 로짓값 : 3, B 항목의 로짓값 : 1 => A 항목을 답으로 추정할 확률이 B 보다 $e^(3-1) = e^2 = 7.39$ 배 더 크다
    
    ex2) A 항목의 로짓값 : 36.5, B 항목의 로짓갑 : 34.5 => A 항목을 답으로 추정할 확률이 B 보다 $e^(36.5-34.5) = e^2 = 7.39$ 배 더 크다

선택 분류 자체만 생각한다면 로짓값들을 구하는 것으로 충분하며 굳이 이를 확률로 변환하여 확률 분포를 따질 필요가 없다. 로짓값이 크면 확률도 크기 때문에 가장 큰 로짓값을 갖는 항목을 선택하면 가장 높은 확률을 갖는 항목을 선택하는 것과 동일하다. 하지만 로짓값만으로는 마땅한 학습 방법을 찾기 어려웠다.

이런 상황에서 사용한 개념이 확률 분포와 교차 엔트로피 개념이다. 선택 분류 문제에서도 신경망이 추정한 확률 분포와 정답이 나타내는 확률 분포 사이의 교차 엔트로피를 구할 수 있고 이를 손실 함수 삼아 학습을 수해앟여 신경망의 선택을 점점 더 정답에 근접시킬 수 있다.

이에 따라 복수의 후보 항목들에 대한 로짓값 벡터를 확률 분포 벡터로 변환하는 함수와 이렇게 구해진 확률 분포와 정답에 나타난 확률 분포 사이의 교차 엔트로피를 계산해주는 함수가 필요하다. 소프트맥스 함수와 소프트맥스 교차 엔트로피 함수가 선택 분류 문제를 해결하기 위해 사용된다.

## Softmax(소프트맥스) 함수

소프트맥스 함수는 로짓값 벡터를 확률 분포 벡터로 변환해주는 비선형 함수다.

(소프트 맥스 함수를 사용해야 하는 이유)

로짓값들은 후보 항들에 대한 선택 확률을 로그 척도로 표현한 것이기 때문에 신경망이 어떤 후보를 골랐는지는 굳이 로그 계산을 하지 않아도 로짓값 벡터에서 최대항이 어느 것인지만 확인하면 바로 알 수 있다. 또한 서낵 분류를 위한 손실 함숫값도 이 로짓값 벡터를 사용하면 바로 계산할 수 있다. 굳이 확률값으로 변환하지 않고도 로짓값만으로도 순전파의 모든 처리가 가능한 것이다.
하지만 두 가지 이유에서 로짓값 벡터를 확률값 벡터, 즉 확률 분포 벡터로 변환해주는 소프트 맥스 함수가 필요하다. 소프트맥스 함수의 출력은 확률 분포, 즉 각 후보 항목이 정답일 확률값들로 구성된 벡터이다. 따라서 출력의 형태는 스칼라값이 아니라 입력 벡터와 동인한 크기를 갖는 벡터이다. 확률의 성질상 벡터의 각 성분은 0 이상 1 이하의 값을 가져야 하며, 전체 벡터 성분의 합은 정확하게 1이 되어야 한다. 물론 확률 분포를 나타내는 이 출력 벡터에는 입력 로짓 벡터에 로그값으로 표현되어 있었던 상대적 가능성 비율이 더 이상 로그 표현이 아닌 확률값으로 변환되어 반영되어 있어야 한다.
소프트맥스 함수의 성질들을 이용하여 소프트맥스 함수의 일반식을 구하면 아래와 같다.
$$y_i = \frac{e^{x_i}}{e^{x_1}+...+e^{x_n}}$$
하지만 이 식은 계산 과정에 오류를 일으킬 수 있기 때문에 실제 소프트맥스 함숫값을 계산할 때는 $x_i$ 가운데 최댓값 $x_k$를 찾은 후 아래의 변형된 식을 사용하여 계산한다.
$$y_i = \frac{e^{x_i-x_k}}{e^{x_1-x_k}+...+e^{x_n-x_k}}$$

### 소프트맥스 함수 사용 예시
ex) 전문가가 로그 척도의 우승 가능성으로 제시한 로짓값 벡터(2.0 1.0 1.2 0.7)은 신경망이 출력해낸 로짓값 벡터에 해당한다. 로짓값은 상대적인 값이기에 원래 나타내려는 값을 추적하자면 어떤 기준이 필요하다. 일단 로짓값이 0일 때의 확률을 기준으로 생각해 보면 각 로짓값이 실제 나타내는 확률 이 기준 로짓값 0이나타내는 확률의 비율을 구해보면 아래와 같다.
$$e^2.0 : e^1.0 : e^1.2 : e^0.7 = 7.39 : 2.72 : 3.32 : 2.01$$
이 비율을 유지하면서 전체 합이 1이 되는 비율을 구하면 아래와 같다.
$$0.479 : 0.176 : 0.215 : 0.130$$

소프트맥스 함수에 입력 벡터가 주어졌다고 할 때 각 $x_i$ 로짓값에 대응하는 실제 확률은 $e^x_i$에 비례하는 값이며, 전체 경우의 확률 합이 1이여야 한다는 점을 활용하여 각 $x_i$에 대응하는 확률을 구하는 함수이다.
즉 소프트맥스 함수는 입력벡터 ($x_1$, ... $x_n$)에 대해 출력 벡터 ($\frac{e^{x_1}}{e^{x_1}+...+e^{x_n}}$, ... $\frac{e^{x_n}}{e^{x_1}+...+e^{x_n}}$)를 출력하는 함수다.

### 소프트맥스 변형식이 필요한 이유와 도출 과정

앞서 정의된 식을 이용해 소프트맥스 함수를 계산하면 범위 제약이 없는 $x_i$ 값들이 제한 없이 커지는 바람에 $e^x_i$ 값을 계산할 때 오버플로 오류가 일어날 가능성이 있다. 또한 모든 값이 절대값이 아주 큰 음수로 쏠리면 분자와 분모가 모두 0에 지나치게 가까워지면서 오류가 발생할 가능성도 있다. 

이러한 문제를 해결하기 위해서 $x_i$의 최댓값을 $x_k$라 할때 분자와 분모를 동시에 $e^x_k$로 나누는 방법이 이용된다.

소프트맥스 변형식을 사용하면서 아래와 같은 이점을 얻을 수 있다.
    
    1. 지수 함수 계산에서 지수가 모두 0 이하라서 지수 함수 계산에서 오버플로가 일어나지 않는다.
    2. 최댓값이 0으로 바뀌는 바람에 최댓값에 해당하는 항의 비율이 1이 되었고, 이에 따라 0 나누기 오류가 발생하지 않는다.

### 소프트맥스 함수의 편미분
소프트맥수 함수는 벡터를 입력받아 벡터를 출력한다. 이 때문에 벡터 성분 간의 편미분이 일대일이나 다대일 관계가 아닌 다대다 관계가 되면서 편미분 계산 과정이 상당히 복잡해진다. 그런데 이 복잡한 편미분 계산 과정은 실제로 딥러닝 학습에 거의 이용되지 않는다. 편미분을 사용하는 역전파에서 막상 편미분 대상을 소프트맥스 함수가 아니라 손실 함수를 구해주는 소프트맥스 교차 엔트로피이기 때문이다.

### 소프트맥스 교차 엔트로피
두 확률 분포 사이의 교차 엔트로피의 정의는 아래와 같다.
$$H(P, Q) = - \sum p_i\log{q_i}$$
로짓 벡터 $a_1$, ... $a_n$과 정답 벡터 $y_1$, ... $y_n$이 주어졌다고 할 때 교차 엔트로피 H(P, Q)를 구하려면 정답 벡터 $y_1$, ... $y_n$을 확률 분포 P로 감고 로짓 벡터 $a_1$, ... $a_n$에 소프트맥스 함수를 적용하여 얻어낸 확률 분포를 Q로 삼아 교차 엔트로피 정의식에 따라 계산하면 된다.

P와 Q를 이렇게 정하는 것은 실은 계산상의 문제 때문이다. 많은 데이터셋에서 정답 벡터는 다양한 확률값을 갖는 확률 분포가 아닌 선택될 원소 하나만 1이고 나머지가 모두 0인 one-hot vector 형태로 표현되다. 그런데 원솟값 모두가 0 혹인 1인 one hot 벡터 성분들이 log 연산에 이용되면 1은 변별력 없는 0 값만 생성하고 0은 $-\infty$ 값을 결과로 하는 오류를 일으키게 된다. 이 때문에 정답 벡터를 로그 연산의 대상이 되지 않는 확률 분포 P로 삼는 것이 유리하다.

이때 로짓 벡터에 소프트맥스 함수를 적용한 결과를 Q로 지정하면 확률의 성질상 모든 원솟값이 항상 0보다 크고1보다 작기에 log 연산에 사용되어도 아무 문제가 없어보인다. 하지만 다른 로짓값들에 비해 유난히 작은 로짓값 $a_i$가 잇따면, 소프트맥스 적용 결과 얻어지는 확률값이 매우 작아 0에 가까운 값이 될 수 있고, 심지어 실수 표현 범위 문제로 인해 0으로 표현되는 경우까지 생길 수 있다. 그런데 이런 값에 로그 함수가 적용되면 함수값이 $-\infty$로 폭주하는 계산 오류가 발생하게 된다.

이를 해결하기 위한 방법은 아주 작은 양수값 $\epsilon$을 도입하여 교차 엔트로피 정의식을 아래와 같이 고쳐서 사용하는 것이다.
$$H(P, Q) = -\sum p_i\log{q_i} \approx - \sum p_i \log{q_i + \epsilon}$$

### 소프트맥스 교차 엔트로피의 편미분

신경망이 추정한 로짓 벡터가 $x_1$, ... $x_n$이고 데이터로 주어진 정답 벡터가 $p_1$, ... $p_n$ 이라고 하자. 이 경우 소프트맥스 함수를 이용하면 로짓 벡터 $x_1$, ..., $x_n$로부터 추정 확률 분포 $q_1$, ... $q_n$을 구할 수 있다. 이 때 소프트맥스 교차 엔트로피의 편미분은 아래와 같은 식으로 구해진다.
$${\partial H \over \partial x_i }= q_i - p_i$$

### 시그모이드 함수와 소프트맥스 함수의 관계
시그모이드 함수는 두 후보 항목을 갖는 소프트맥스 함수의 입력 벡터 $x_1$, $x_2$에서 $x_2 = 0$으로 설정해 입력 변수를 하나로 줄인 함수에 해당한다. 소프트맥스 함수를 조금 손질하여 적용하면 바로 시그모이드 함수가 된다.

시그모이드 함수에서 $x_2$ 값을 0으로 고정시켜놓더라도 이 값이 확률값이 아니라 로짓값이기 때문에 계산에 전혀 문제가 없다. 또한 전체 경우의 확률 총합은 언제나 1이기 때문에 이진 판단에서 참일 확률만 알면 거짓일 확률을 바로 알 수 있기에 따로 따지지 않고 $x_2$값을 적당히 고정시켜도 문제가 없다.